In [19]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16,decode_predictions,preprocess_input
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,Flatten

In [21]:
dataGen=ImageDataGenerator(rescale=1/255,validation_split=0.2)

In [22]:
train_dir='/kaggle/input/automobilepartsindentification/Automobile-parts'
train_itt=dataGen.flow_from_directory(directory=train_dir,batch_size=64, 
                                            target_size=(224, 224),subset='training')
test_itt=dataGen.flow_from_directory(directory=train_dir,batch_size=64, 
                                          target_size=(224, 224),subset='validation')

In [23]:
model=VGG16(include_top=False,input_shape=(224,224,3))
for layer in model.layers:
    # specify the layers loaded are not trainable
    model.trainable=False

In [24]:
flat_layer=Flatten()(model.output)
fc_layer1=Dense(512,activation='relu')(flat_layer)
fc_layer_2=Dense(256,activation='relu')(fc_layer1)
fc_layer_softmax=Dense(14,activation='softmax')(fc_layer_2)
model=Model(inputs=model.input, outputs=fc_layer_softmax)

In [25]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(train_itt,validation_data=test_itt,epochs=10)

In [26]:
losses = pd.DataFrame(model.history.history)

In [27]:
plt.plot(losses['loss'])
plt.plot(losses['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('no of epochs')
plt.legend(['train','test'],loc='upper left')
plt.show()

In [28]:
plt.plot(losses['accuracy'])
plt.plot(losses['val_accuracy'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('no of epochs')
plt.legend(['train','test'],loc='upper left')
plt.show()

In [29]:
class_label= test_itt.class_indices
class_label

In [30]:
from tensorflow.keras.preprocessing.image import load_img,img_to_array

In [31]:
image=load_img('/kaggle/input/automobilepartsindentification/Automobile-parts/bearing/image (174).jpg',
               target_size=(224,224))
plt.imshow(image)
plt.show()

In [41]:
def prediction(path):
    image=load_img(path,target_size=(224,224))
    plt.imshow(image)
    plt.show()
    image=img_to_array(image)
    image=image.reshape(1,224,224,3)
    result=model.predict(image)
    result=np.argmax(result)
    return print([key for key in class_label][result])

In [42]:
prediction('/kaggle/input/automobilepartsindentification/Automobile-parts/bearing/image (174).jpg')